In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import regex as re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests

C:\Users\vella\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(r"C:\Users\vella\Final year project\Data\Sentiments of Elon tweets\ElonTweets(Sentiment).csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,Datetime,Tweet Id,Text,Username,location,reply count,retweet count,like count,language,Twitter Access Point,Follower Count,Friends Count,verified,Date,mentions,sentiment
0,0,0,2022-09-29 20:22:46+00:00,1575581872913907712,@MuskUniversity True,elonmusk,NaN,377,127,3007,en,Twitter for iPhone,107414837,122,True,2022-09-29,MuskUniversity,"['neutral', 0.6005306]"
1,1,1,2022-09-29 15:35:09+00:00,1575509494510993408,Needs be able to get from Starbase to South Pa...,elonmusk,NaN,1604,1416,33556,en,Twitter for iPhone,107414837,122,True,2022-09-29,_,"['neutral', 0.8452575]"
2,2,2,2022-09-29 15:32:42+00:00,1575508878300618752,@WholeMarsBlog Off-label use 🤣,elonmusk,NaN,754,492,23482,en,Twitter for iPhone,107414837,122,True,2022-09-29,WholeMarsBlog,"['neutral', 0.7530303]"
3,3,3,2022-09-29 15:31:12+00:00,1575508498430820352,Cybertruck will be waterproof enough to serve ...,elonmusk,NaN,12870,9732,124276,en,Twitter for iPhone,107414837,122,True,2022-09-29,_,"['positive', 0.6118259]"
4,4,4,2022-09-29 15:28:10+00:00,1575507735298207744,@phibetakitten Submarines use electric motors ...,elonmusk,NaN,509,347,8189,en,Twitter for iPhone,107414837,122,True,2022-09-29,phibetakitten,"['neutral', 0.85970926]"


In [3]:
df1 = df.iloc[:,[4,9,15,17]]

In [4]:
#Extracting all required values into respective lists
tweets = df1['Text']

In [5]:
#STEP 1: Removing mentions and urls from tweet messages
tweets_without_mention = []
for message in tweets:
    tweets_without_mention.append(re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "",message))

In [6]:
#STEP 2: Removing consecutive non-ascii characters
tweets_without_non_ascii = []
for message in tweets_without_mention:
    tweets_without_non_ascii.append(re.sub(r'[^\x00-\x7F]+',' ', message))

In [7]:
#STEP 3: Tokenization and remove punctuations
tweets_tokens = []
for message in tweets_without_non_ascii:
    words = word_tokenize(message)
    words_without_punctuation = [word for word in words if word.isalnum()]
    tweets_tokens.append(words_without_punctuation)

In [8]:
#STEP 4: Removing stop words
stop_words = set(stopwords.words('english'))
tweets_without_stop_words = [] 
  
for i in tweets_tokens:
    temp = []
    for j in i:
        if j not in stop_words: 
            temp.append(j)
    tweets_without_stop_words.append(temp)

In [9]:
#STEP 5: Removing emoticons from tweets
tweets_without_emoticons=[]
for message in tweets_without_stop_words:
    temp=[]
    for word in message:
        to_remove_pattern = re.compile(pattern = "["
                                       u"\U0001F600-\U0001F64F"  # emoticons
                                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                       "]+", flags = re.UNICODE)
        temp.append(to_remove_pattern.sub(r'',word))
    tweets_without_emoticons.append(temp)

In [10]:
#STEP 6: Final processed tweet messages
processed_tweets= []
for message in tweets_without_emoticons:
    s = " "
    s = s.join(message)
    processed_tweets.append(s)

In [11]:
df1['Processed_tweet'] = processed_tweets

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1

,Text,like count,Date,sentiment,Processed_tweet
0,@MuskUniversity True,3007,2022-09-29,"['neutral', 0.6005306]",True
1,Needs be able to get from Starbase to South Pa...,33556,2022-09-29,"['neutral', 0.8452575]",Needs able get Starbase South Padre Island req...
2,@WholeMarsBlog Off-label use 🤣,23482,2022-09-29,"['neutral', 0.7530303]",use
3,Cybertruck will be waterproof enough to serve ...,124276,2022-09-29,"['positive', 0.6118259]",Cybertruck waterproof enough serve briefly boa...
4,@phibetakitten Submarines use electric motors ...,8189,2022-09-29,"['neutral', 0.85970926]",Submarines use electric motors water
...,...,...,...,...,...
17597,That was a total non sequitur btw,50,2011-12-03,"['negative', 0.5067324]",That total non sequitur btw
17598,"Great Voltaire quote, arguably better than Twa...",50,2011-12-03,"['positive', 0.55860823]",Great Voltaire quote arguably better Twain Hea...
17599,I made the volume on the Model S http://t.co/w...,74,2011-12-01,"['neutral', 0.6384011]",I made volume Model S go 11 Now I need work mi...
17600,Went to Iceland on Sat to ride bumper cars on ...,189,2011-12-01,"['positive', 0.9733198]",Went Iceland Sat ride bumper cars ice No count...


In [29]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vella\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [30]:
df1['scores'] = df1['Processed_tweet'].apply(lambda Processed_tweet: sid.polarity_scores(Processed_tweet))
df1.head()

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Text,like count,Date,sentiment,Processed_tweet,scores
0,@MuskUniversity True,3007,2022-09-29,"['neutral', 0.6005306]",True,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."
1,Needs be able to get from Starbase to South Pa...,33556,2022-09-29,"['neutral', 0.8452575]",Needs able get Starbase South Padre Island req...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,@WholeMarsBlog Off-label use 🤣,23482,2022-09-29,"['neutral', 0.7530303]",use,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Cybertruck will be waterproof enough to serve ...,124276,2022-09-29,"['positive', 0.6118259]",Cybertruck waterproof enough serve briefly boa...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,@phibetakitten Submarines use electric motors ...,8189,2022-09-29,"['neutral', 0.85970926]",Submarines use electric motors water,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


# considering compound score only

In [31]:
df1['compound']  = df1['scores'].apply(lambda score_dict: score_dict['compound'])

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df1.head(50)

,Text,like count,Date,sentiment,Processed_tweet,scores,compound
0,@MuskUniversity True,3007,2022-09-29,"['neutral', 0.6005306]",True,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4215
1,Needs be able to get from Starbase to South Pa...,33556,2022-09-29,"['neutral', 0.8452575]",Needs able get Starbase South Padre Island req...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,@WholeMarsBlog Off-label use 🤣,23482,2022-09-29,"['neutral', 0.7530303]",use,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
3,Cybertruck will be waterproof enough to serve ...,124276,2022-09-29,"['positive', 0.6118259]",Cybertruck waterproof enough serve briefly boa...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,@phibetakitten Submarines use electric motors ...,8189,2022-09-29,"['neutral', 0.85970926]",Submarines use electric motors water,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
5,Works on land too,11234,2022-09-29,"['neutral', 0.8428305]",Works land,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
6,As well as advanced chip &amp; supercomputer e...,13179,2022-09-29,"['neutral', 0.67935413]",As well advanced chip amp supercomputer engine...,"{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'comp...",0.4767
7,"Note, this event is meant for recruiting AI &a...",32393,2022-09-29,"['neutral', 0.59770757]",Note event meant recruiting AI amp robotics en...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
8,@stevenmarkryan Pretty much. AI/robotics engin...,8231,2022-09-29,"['neutral', 0.53000236]",Pretty much engineers understand problems need...,"{'neg': 0.174, 'neu': 0.323, 'pos': 0.503, 'co...",0.6249
9,@WholeMarsBlog @Gfilche There will be lots of ...,4508,2022-09-29,"['positive', 0.92369086]",There lots technical detail amp cool hardware ...,"{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'comp...",0.3182


In [33]:
df1['Sentiment'] = df1['like count']*(df1['compound'])

C:\Users\vella\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
df1.head(60)

,Text,like count,Date,sentiment,Processed_tweet,scores,compound,Sentiment
0,@MuskUniversity True,3007,2022-09-29,"['neutral', 0.6005306]",True,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4215,1267.4505
1,Needs be able to get from Starbase to South Pa...,33556,2022-09-29,"['neutral', 0.8452575]",Needs able get Starbase South Padre Island req...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
2,@WholeMarsBlog Off-label use 🤣,23482,2022-09-29,"['neutral', 0.7530303]",use,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
3,Cybertruck will be waterproof enough to serve ...,124276,2022-09-29,"['positive', 0.6118259]",Cybertruck waterproof enough serve briefly boa...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
4,@phibetakitten Submarines use electric motors ...,8189,2022-09-29,"['neutral', 0.85970926]",Submarines use electric motors water,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
5,Works on land too,11234,2022-09-29,"['neutral', 0.8428305]",Works land,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
6,As well as advanced chip &amp; supercomputer e...,13179,2022-09-29,"['neutral', 0.67935413]",As well advanced chip amp supercomputer engine...,"{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'comp...",0.4767,6282.4293
7,"Note, this event is meant for recruiting AI &a...",32393,2022-09-29,"['neutral', 0.59770757]",Note event meant recruiting AI amp robotics en...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
8,@stevenmarkryan Pretty much. AI/robotics engin...,8231,2022-09-29,"['neutral', 0.53000236]",Pretty much engineers understand problems need...,"{'neg': 0.174, 'neu': 0.323, 'pos': 0.503, 'co...",0.6249,5143.5519
9,@WholeMarsBlog @Gfilche There will be lots of ...,4508,2022-09-29,"['positive', 0.92369086]",There lots technical detail amp cool hardware ...,"{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'comp...",0.3182,1434.4456


In [35]:
df2 = df1.dropna()
df2 = df2.reset_index()

In [36]:
df2.head(265)

,index,Text,like count,Date,sentiment,Processed_tweet,scores,compound,Sentiment
0,0,@MuskUniversity True,3007,2022-09-29,"['neutral', 0.6005306]",True,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4215,1267.4505
1,1,Needs be able to get from Starbase to South Pa...,33556,2022-09-29,"['neutral', 0.8452575]",Needs able get Starbase South Padre Island req...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
2,2,@WholeMarsBlog Off-label use 🤣,23482,2022-09-29,"['neutral', 0.7530303]",use,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
3,3,Cybertruck will be waterproof enough to serve ...,124276,2022-09-29,"['positive', 0.6118259]",Cybertruck waterproof enough serve briefly boa...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
4,4,@phibetakitten Submarines use electric motors ...,8189,2022-09-29,"['neutral', 0.85970926]",Submarines use electric motors water,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
260,260,"@CSI_Starbase Agreed, we recently made this ch...",5510,2022-08-29,"['neutral', 0.53914475]",Agreed recently made change amp many others An...,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp...",0.7506,4135.8060
261,261,This will happen again – just a matter of time...,55846,2022-08-28,"['neutral', 0.46470997]",This happen matter time,"{'neg': 0.0, 'neu': 0.732, 'pos': 0.268, 'comp...",0.0258,1440.8268
262,262,Godspeed Artemis! https://t.co/8rZSQPamBf,51847,2022-08-28,"['positive', 0.8628223]",Godspeed Artemis,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0000
263,263,@RenataKonkoly @Kristennetten @culturaltutor h...,2983,2022-08-28,"['neutral', 0.6296353]",,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0.0000,0.0000
